In [1]:
from PIL import Image
import numpy as np
import pickle
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from tqdm import tqdm

c:\users\sasha\desktop\thesis\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    return preprocess_input(x, mode='tf')

In [3]:
with open("train.p", "rb") as train_pickle:
    train_images, train_labels = pickle.load(train_pickle)

In [4]:
with open("valid.p", "rb") as valid_pickle:
    valid_images, valid_labels = pickle.load(valid_pickle)

In [5]:
base_model = ResNet50(weights='imagenet')

base_model.summary()

102858752/102853048 [==============================] - 104s 1us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
________________________________________

In [6]:
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

In [7]:
extracted_features_train = []
for img in tqdm(train_images):
    preprocessed_image = preprocess_image(img)
    extracted_features_train.append(model.predict(preprocessed_image)[0])

100%|████████████████████████████████████████████████████████████████████████████| 6514/6514 [2:08:56<00:00,  1.19s/it]


In [8]:
with open("train_resnet50.p", "wb") as train_pickle:
    pickle.dump((np.array(extracted_features_train), train_labels), train_pickle)

In [9]:
extracted_features_valid = []
for img in tqdm(valid_images):
    preprocessed_image = preprocess_image(img)
    extracted_features_valid.append(model.predict(preprocessed_image)[0])

100%|████████████████████████████████████████████████████████████████████████████████| 814/814 [09:13<00:00,  1.47it/s]


In [10]:
with open("valid_resnet50.p", "wb") as valid_pickle:
    pickle.dump((np.array(extracted_features_valid), valid_labels), valid_pickle)